In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

# %pip install transformers
# %pip install kobert-transformers

# %pip install scikit-learn
# %pip install tqdm
# %pip install numpy
# %pip install pandas

# %pip install sentencepiece
# %pip install tokenizers

# %pip install accelerate
# %pip install bitsandbytes  # (필요 없으면 생략 가능)

# %pip install python-dotenv
# %pip install hf_transfer

In [ ]:
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from kobert_transformers import get_kobert_model, get_tokenizer
from sklearn.metrics import f1_score
from tqdm import tqdm

# ============================================================
# 1) 라벨 정의
# ============================================================
# LABEL_LIST = [
#     "MONSTER_GUIDE",
#     "EVENT_GUIDE",
#     "DUNGEON_NAVIGATOR",
#     "INTERACTION_HANDLER",
#     "USAGE_GUIDE",
#     "SMALLTALK",
#     "UNKNOWN_INTENT"
# ]
LABEL_LIST = [
    "INVENTORY_ITEM_USE",
    "LIGHT_ON_ROOM",
    "LIGHT_OFF_ROOM",
    "MOVE_NEXT_ROOM",
    "NONE",
]

label2idx = {l: i for i, l in enumerate(LABEL_LIST)}
idx2label = {i: l for l, i in label2idx.items()}


In [ ]:
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0

    for batch in tqdm(loader, desc="Train"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

def eval_epoch(model, loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Eval"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            total_loss += loss.item()

            probs = torch.sigmoid(logits).cpu()
            preds = (probs > 0.5).int()

            all_preds.extend(preds.numpy())
            all_labels.extend(labels.cpu().numpy())

    micro = f1_score(all_labels, all_preds, average="micro", zero_division=0)
    macro = f1_score(all_labels, all_preds, average="macro", zero_division=0)

    return total_loss / len(loader), micro, macro


def predict(text, model, tokenizer, device="cpu", threshold=0.5):
    model.eval()
    with torch.no_grad():
        inputs = tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=64,
            return_tensors="pt"
        )
        input_ids = inputs["input_ids"].to(device)
        attention_mask = inputs["attention_mask"].to(device)

        logits = model(input_ids, attention_mask)
        probs = torch.sigmoid(logits).cpu().squeeze(0)

        pred_idx = (probs > threshold).nonzero().flatten().tolist()
        pred_labels = [LABEL_LIST[i] for i in pred_idx]

        return pred_labels, probs.tolist()

In [ ]:
from lab.fairy.KoBertMultiLabelClassifier import KoBertMultiLabelClassifier
from lab.fairy.IntentDataset import IntentDataset
# DATA_PATH = "fairy_dungeon_intent_dataset_total.json"
DATA_PATH = "fairy_interaction_intent_output.json"
# MODEL_NAME = "kobert_intent_model.pt"
MODEL_NAME = "kobert_interaction_intent_model.pt"
# device = torch.device("cuda")
device = torch.device("cpu")
tokenizer = get_tokenizer()

dataset = IntentDataset(DATA_PATH, tokenizer, label2idx)
train_size = int(len(dataset) * 0.9)
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=16)

model = KoBertMultiLabelClassifier(num_labels=len(LABEL_LIST)).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

EPOCHS = 6

for epoch in range(EPOCHS):
    print(f"\n===== Epoch {epoch+1} =====")
    train_loss = train_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, micro, macro = eval_epoch(model, val_loader, criterion, device)

    print(f"Train Loss: {train_loss:.4f}")
    print(f"Val Loss:   {val_loss:.4f}")
    print(f"Micro F1:   {micro:.4f}")
    print(f"Macro F1:   {macro:.4f}")

checkpoint = {
    "model_state_dict": model.state_dict(),
    "idx2label": idx2label,
    "label2idx": label2idx,
}

# 모델 저장
torch.save(checkpoint, MODEL_NAME)

print(f"\n모델 저장 완료 → {MODEL_NAME}")
# 테스트 예측
test = "한손검을 사용하고 방에 불좀 켜줘"
labels, probs = predict(test, model, tokenizer, device=device)
print("\n[테스트 입력]", test)
print("[예측 결과]", labels)


In [ ]:
from huggingface_hub import HfApi
from dotenv import load_dotenv
import os
load_dotenv()


HF_TOKEN = os.getenv("HF_TOKEN")
REPO_ID = "JINSUP/ProjectML-Models"
FILE_NAME_IN_REPO = "fairy_interaction_intent_kobert_classifier.pt"



# 저장소 자동 생성
api = HfApi()

api.upload_file(
    path_or_fileobj=MODEL_NAME,
    repo_id=REPO_ID,
    path_in_repo=FILE_NAME_IN_REPO,
    commit_message="update kobert interaction intent classifier model",
    token=HF_TOKEN
)


print("업로드 완료:", REPO_ID)